In [41]:
import sys
print(sys.executable)

C:\Users\maxim\anaconda3\python.exe


In [42]:
import psycopg

conn = psycopg.connect(
    host="localhost",
    port='5432',
    dbname="postgres",
    user="postgres",
    password="123")

In [43]:
cur = conn.cursor()

In [44]:
print('PostgreSQL database version:')
cur.execute('SELECT version()')

db_version = cur.fetchone()
print(db_version)

PostgreSQL database version:
('PostgreSQL 17.6 (Debian 17.6-1.pgdg13+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 14.2.0-19) 14.2.0, 64-bit',)


In [45]:
cur.execute("DROP TABLE IF EXISTS public.mta_ridership CASCADE;")
cur.execute("DROP TABLE IF EXISTS public.mta_zones CASCADE;")
cur.execute("DROP TABLE IF EXISTS public.citibike_trips CASCADE;")
cur.execute("DROP TABLE IF EXISTS public.citibike_zones CASCADE;")
cur.execute("DROP TABLE IF EXISTS public.taxi_ridership CASCADE;")
cur.execute("DROP TABLE IF EXISTS public.taxi_zones CASCADE;")
cur.execute("DROP TABLE IF EXISTS public.citibike_stations CASCADE;")
cur.execute("DROP TABLE IF EXISTS public.mta_stations CASCADE;")

conn.commit()

print("All tables dropped.")

All tables dropped.


In [46]:
table_sqls = [

    # Citibike zones
    """
    CREATE TABLE IF NOT EXISTS public.citibike_zones (
        station_id VARCHAR PRIMARY KEY,
        zip_code VARCHAR,
        LocationID VARCHAR,
        Zone VARCHAR
    );
    """,

    # Citibike trips
    """
    CREATE TABLE IF NOT EXISTS public.citibike_trips (
        trip_id SERIAL PRIMARY KEY,
        station_id VARCHAR NOT NULL REFERENCES public.citibike_zones(station_id),
        station_name VARCHAR,
        date VARCHAR,
        hour INT
    );
    """,

    # MTA zones
    """
    CREATE TABLE IF NOT EXISTS public.mta_zones (
        station_clean VARCHAR PRIMARY KEY,
        zip_code VARCHAR,
        LocationID VARCHAR,
        Zone VARCHAR
    );
    """,

    # MTA ridership
    """
    CREATE TABLE IF NOT EXISTS public.mta_ridership (
        ridership_id SERIAL PRIMARY KEY,
        station_clean VARCHAR NOT NULL REFERENCES public.mta_zones(station_clean),
        date VARCHAR,
        hour INT,
        avg_ridership FLOAT
    );
    """,

    # Taxi zones
    """
    CREATE TABLE IF NOT EXISTS public.taxi_zones (
        LocationID VARCHAR PRIMARY KEY,
        Borough VARCHAR,
        Zone VARCHAR,
        service_zone VARCHAR
    );
    """,

    # Taxi ridership
    """
    CREATE TABLE IF NOT EXISTS public.taxi_ridership (
        ridership_id SERIAL PRIMARY KEY,
        date VARCHAR,
        hour INT,
        LocationID VARCHAR NOT NULL REFERENCES public.taxi_zones(LocationID),
        zone VARCHAR,
        ridership FLOAT
    );
    """
]

for sql_text in table_sqls:
    cur.execute(sql_text)
    print("Executed:", sql_text.split("(")[0].strip())

conn.commit()
print("Committed DDL statements.")

Executed: CREATE TABLE IF NOT EXISTS public.citibike_zones
Executed: CREATE TABLE IF NOT EXISTS public.citibike_trips
Executed: CREATE TABLE IF NOT EXISTS public.mta_zones
Executed: CREATE TABLE IF NOT EXISTS public.mta_ridership
Executed: CREATE TABLE IF NOT EXISTS public.taxi_zones
Executed: CREATE TABLE IF NOT EXISTS public.taxi_ridership
Committed DDL statements.


In [47]:
cur.execute("""
SELECT table_schema, table_name
FROM information_schema.tables
WHERE table_schema='public'
ORDER BY table_name;
""")

for row in cur.fetchall():
    print(row)

('public', 'citibike_trips')
('public', 'citibike_zones')
('public', 'mta_ridership')
('public', 'mta_zones')
('public', 'taxi_ridership')
('public', 'taxi_zones')


In [48]:
time_bucket_sql = """
CREATE OR REPLACE FUNCTION time_bucket(hour INT) RETURNS VARCHAR AS $$
BEGIN
  RETURN CASE
    WHEN hour >= 0 AND hour < 6 THEN 'Night'
    WHEN hour >= 6 AND hour < 12 THEN 'Morning'
    WHEN hour >= 12 AND hour < 18 THEN 'Afternoon'
    WHEN hour >= 18 AND hour < 24 THEN 'Evening'
    ELSE 'Unknown'
  END;
END;
$$ LANGUAGE plpgsql IMMUTABLE;
"""

cur.execute(time_bucket_sql)
conn.commit()

print("Created function time_bucket()")

Created function time_bucket()


In [49]:
cur.execute("SELECT time_bucket(3);")
print(cur.fetchone()[0])   # should print 'Night'

cur.execute("SELECT time_bucket(14);")
print(cur.fetchone()[0])   # should print 'Afternoon'

Night
Afternoon


In [50]:
cur.execute("""
    SELECT table_schema, table_name
    FROM information_schema.tables
    WHERE table_schema NOT IN ('information_schema', 'pg_catalog')
    ORDER BY table_schema, table_name;
""")

tables = cur.fetchall()

print("Tables in database:")
for schema, name in tables:
    print(f"{schema}.{name}")

Tables in database:
public.citibike_trips
public.citibike_zones
public.mta_ridership
public.mta_zones
public.taxi_ridership
public.taxi_zones


In [51]:
import pandas as pd

citibike_zones_df = pd.read_csv("citibike_zones.csv")
citibike_trips_df = pd.read_csv("citibike_rider.csv")
mta_zones_df = pd.read_csv("station_zones.csv")
mta_ridership_df = pd.read_csv("mta_final.csv")
taxi_zones_df = pd.read_csv("taxi_zones.csv")
taxi_ridership_df = pd.read_csv("taxi_riderships.csv")

citibike_zones_df.head(), citibike_trips_df.head(), mta_zones_df.head(), mta_ridership_df.head(), taxi_zones_df.head(), taxi_ridership_df.head()

(  station_id  zip_code  LocationID Zone
 0      HB203      7030         NaN  NaN
 1      HB608      7030         NaN  NaN
 2      JC018      7306         NaN  NaN
 3      JC008      7310         NaN  NaN
 4      JC115      7302         NaN  NaN,
    hour   date           station_name station_id
 0     0  01-01  Bloomfield St & 15 St      HB203
 1     0  01-01        2 St & Park Ave      HB608
 2     0  01-01      5 Corners Library      JC018
 3     0  01-01  Bloomfield St & 15 St      HB203
 4     0  01-01           Newport Pkwy      JC008,
                 station_clean  zip_code  LocationID                  Zone
 0                        1 Av     10009           4         Alphabet City
 1                      103 St     10029          74     East Harlem North
 2                      110 St     10026          42  Central Harlem North
 3                      116 St     10025         151      Manhattan Valley
 4  116 St-Columbia University     10027         166   Morningside Heights,
 

In [52]:
%%time
# ----------------------------------
# 2. INSERT FUNCTION (psycopg3)
# ----------------------------------
def insert_dataframe(conn, df, table_name, columns):
    with conn.cursor() as cur:
        records = [tuple(row[col] for col in columns) for _, row in df.iterrows()]
        placeholders = ", ".join(["%s"] * len(columns))
        colnames = ", ".join(columns)
        sql = f"INSERT INTO {table_name} ({colnames}) VALUES ({placeholders});"
        cur.executemany(sql, records)
    conn.commit()
    print(f"Inserted {len(df)} rows into {table_name}.")


# ----------------------------------
# 3. INSERT INTO ALL TABLES
# ----------------------------------
insert_dataframe(
    conn,
    citibike_zones_df,
    "public.citibike_zones",
    ["station_id", "zip_code", "LocationID", "Zone"]
)

insert_dataframe(
    conn,
    citibike_trips_df,
    "public.citibike_trips",
    ["station_id", "station_name", "date", "hour"]
)

insert_dataframe(
    conn,
    mta_zones_df,
    "public.mta_zones",
    ["station_clean", "zip_code", "LocationID", "Zone"]
)

insert_dataframe(
    conn,
    mta_ridership_df,
    "public.mta_ridership",
    ["station_clean", "date", "hour", "avg_ridership"]
)

insert_dataframe(
    conn,
    taxi_zones_df,
    "public.taxi_zones",
    ["LocationID", "Borough", "Zone", "service_zone"]
)

insert_dataframe(
    conn,
    taxi_ridership_df,
    "public.taxi_ridership",
    ["date", "hour", "LocationID", "Zone", "ridership"]
)


print("All data inserted successfully.")

Inserted 488 rows into public.citibike_zones.
Inserted 2060077 rows into public.citibike_trips.
Inserted 87 rows into public.mta_zones.
Inserted 763781 rows into public.mta_ridership.
Inserted 265 rows into public.taxi_zones.
Inserted 602468 rows into public.taxi_ridership.
All data inserted successfully.
CPU times: total: 5min 15s
Wall time: 5min 24s


In [61]:
query = """
WITH citibike_sums AS (
    SELECT
        cz.Zone,
        ct.date,
        time_bucket(ct.hour) AS bucket,
        COUNT(*) AS citibike_count
    FROM public.citibike_trips ct
    JOIN public.citibike_zones cz
        ON ct.station_id = cz.station_id
    GROUP BY cz.Zone, ct.date, bucket
),

mta_sums AS (
    SELECT
        mz.Zone,
        mr.date,
        time_bucket(mr.hour) AS bucket,
        SUM(mr.avg_ridership) AS mta_count
    FROM public.mta_ridership mr
    JOIN public.mta_zones mz
        ON mr.station_clean = mz.station_clean
    GROUP BY mz.Zone, mr.date, bucket
),

taxi_sums AS (
    SELECT
        tz.Zone,
        tr.date,
        time_bucket(tr.hour) AS bucket,
        SUM(tr.ridership) AS taxi_count
    FROM public.taxi_ridership tr
    JOIN public.taxi_zones tz
        ON tr.LocationID = tz.LocationID
    GROUP BY tz.Zone, tr.date, bucket
),

-- Merge Bike + Subway
bike_subway AS (
    SELECT
        COALESCE(c.Zone, m.Zone) AS Zone,
        COALESCE(c.date, m.date) AS date,
        COALESCE(c.bucket, m.bucket) AS bucket,
        COALESCE(c.citibike_count, 0) AS citibike_count,
        COALESCE(m.mta_count, 0) AS mta_count
    FROM citibike_sums c
    FULL OUTER JOIN mta_sums m
        ON c.Zone = m.Zone
       AND c.date = m.date
       AND c.bucket = m.bucket
),

-- Merge Bike+Subway with Taxi
combined AS (
    SELECT
        COALESCE(bs.Zone, t.Zone) AS Zone,
        COALESCE(bs.date, t.date) AS date,
        COALESCE(bs.bucket, t.bucket) AS bucket,
        COALESCE(bs.citibike_count, 0)
            + COALESCE(bs.mta_count, 0)
            + COALESCE(t.taxi_count, 0) AS total_people
    FROM bike_subway bs
    FULL OUTER JOIN taxi_sums t
        ON bs.Zone = t.Zone
       AND bs.date = t.date
       AND bs.bucket = t.bucket
)

SELECT
    bucket AS time_of_day,
    AVG(total_people)::numeric(12,2) AS avg_people
FROM combined
GROUP BY bucket
ORDER BY
    CASE bucket
        WHEN 'Night' THEN 1
        WHEN 'Morning' THEN 2
        WHEN 'Afternoon' THEN 3
        WHEN 'Evening' THEN 4
    END;
"""

df = pd.read_sql(query, conn)
print(df)

C:\Users\maxim\AppData\Local\Temp\ipykernel_14828\3991775661.py:83: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


  time_of_day  avg_people
0       Night     4016.54
1     Morning    11421.61
2   Afternoon    17404.84
3     Evening    17425.15
4     Unknown      773.79


In [62]:
import psycopg
import pandas as pd
#get_avg_for_hour(date, hour, zone)
def get_avg_for_hour(conn, date: str, hour: int, zone: str):
    """
    Returns total ridership for a specific date, hour, and zone
    across Citibike, MTA, and Taxi.
    """

    query = """
    WITH bike AS (
        SELECT COUNT(*) AS citibike_count
        FROM public.citibike_trips ct
        JOIN public.citibike_zones cz
            ON ct.station_id = cz.station_id
        WHERE ct.date = %s
          AND ct.hour = %s
          AND cz.Zone = %s
    ),
    subway AS (
        SELECT SUM(mr.avg_ridership) AS mta_count
        FROM public.mta_ridership mr
        JOIN public.mta_zones mz
            ON mr.station_clean = mz.station_clean
        WHERE mr.date = %s
          AND mr.hour = %s
          AND mz.Zone = %s
    ),
    taxi AS (
        SELECT SUM(tr.ridership) AS taxi_count
        FROM public.taxi_ridership tr
        JOIN public.taxi_zones tz
            ON tr.LocationID = tz.LocationID
        WHERE tr.date = %s
          AND tr.hour = %s
          AND tz.Zone = %s
    )
    SELECT
        COALESCE(citibike_count,0) +
        COALESCE(mta_count,0) +
        COALESCE(taxi_count,0) AS total_people
    FROM bike, subway, taxi;
    """

    params = (date, hour, zone, date, hour, zone, date, hour, zone)
    df = pd.read_sql(query, conn, params=params)
    return df.iloc[0]["total_people"]

In [63]:
import psycopg
import pandas as pd
#get_avg_by_time_buckets(date, zone)
def get_avg_by_time_buckets(conn, date: str, zone: str):
    """
    Returns average ridership bucketed into
    Night, Morning, Afternoon, and Evening for a given date + zone.
    """

    query = """
    WITH citibike_sums AS (
        SELECT
            time_bucket(ct.hour) AS bucket,
            COUNT(*) AS citibike_count
        FROM public.citibike_trips ct
        JOIN public.citibike_zones cz
            ON ct.station_id = cz.station_id
        WHERE ct.date = %s
          AND cz.Zone = %s
        GROUP BY bucket
    ),
    mta_sums AS (
        SELECT
            time_bucket(mr.hour) AS bucket,
            SUM(mr.avg_ridership) AS mta_count
        FROM public.mta_ridership mr
        JOIN public.mta_zones mz
            ON mr.station_clean = mz.station_clean
        WHERE mr.date = %s
          AND mz.Zone = %s
        GROUP BY bucket
    ),
    taxi_sums AS (
        SELECT
            time_bucket(tr.hour) AS bucket,
            SUM(tr.ridership) AS taxi_count
        FROM public.taxi_ridership tr
        JOIN public.taxi_zones tz
            ON tr.LocationID = tz.LocationID
        WHERE tr.date = %s
          AND tz.Zone = %s
        GROUP BY bucket
    ),
    combined AS (
        SELECT
            COALESCE(c.bucket, m.bucket, t.bucket) AS bucket,
            COALESCE(c.citibike_count,0) +
            COALESCE(m.mta_count,0) +
            COALESCE(t.taxi_count,0) AS total_people
        FROM citibike_sums c
        FULL OUTER JOIN mta_sums m
          ON c.bucket = m.bucket
        FULL OUTER JOIN taxi_sums t
          ON COALESCE(c.bucket, m.bucket) = t.bucket
    )
    SELECT
        bucket,
        AVG(total_people)::numeric(12,2) AS avg_people
    FROM combined
    GROUP BY bucket
    ORDER BY
        CASE bucket
            WHEN 'Night' THEN 1
            WHEN 'Morning' THEN 2
            WHEN 'Afternoon' THEN 3
            WHEN 'Evening' THEN 4
        END;
    """

    params = (date, zone, date, zone, date, zone)
    df = pd.read_sql(query, conn, params=params)
    return df

In [64]:
result = get_avg_for_hour(conn, "06-01", 14, "Midtown South")
print("People at 2 PM:", result)

People at 2 PM: 2430.0


C:\Users\maxim\AppData\Local\Temp\ipykernel_14828\1316344652.py:46: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, params=params)


In [65]:
df = get_avg_by_time_buckets(conn, "06-01", "Midtown South")
print(df)

      bucket  avg_people
0      Night      5660.0
1    Morning      8310.0
2  Afternoon     14534.0
3    Evening     21340.0
4    Unknown      1432.0


C:\Users\maxim\AppData\Local\Temp\ipykernel_14828\3784119502.py:71: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, params=params)


In [66]:
import psycopg
import pandas as pd
"""
Returns:
1) Total people for the specific date + hour + zone
2) Bucketed averages (Night/Morning/Afternoon/Evening)
"""
def get_ridership_info(conn, date: str, hour: int, zone: str):

    # 1) Query for specific hour
    query_specific = """
    WITH bike AS (
        SELECT COUNT(*) AS citibike_count
        FROM public.citibike_trips ct
        JOIN public.citibike_zones cz
            ON ct.station_id = cz.station_id
        WHERE ct.date = %s
          AND ct.hour = %s
          AND cz.Zone = %s
    ),
    subway AS (
        SELECT SUM(mr.avg_ridership) AS mta_count
        FROM public.mta_ridership mr
        JOIN public.mta_zones mz
            ON mr.station_clean = mz.station_clean
        WHERE mr.date = %s
          AND mr.hour = %s
          AND mz.Zone = %s
    ),
    taxi AS (
        SELECT SUM(tr.ridership) AS taxi_count
        FROM public.taxi_ridership tr
        JOIN public.taxi_zones tz
            ON tr.LocationID = tz.LocationID
        WHERE tr.date = %s
          AND tr.hour = %s
          AND tz.Zone = %s
    )
    SELECT
        COALESCE(citibike_count,0) +
        COALESCE(mta_count,0) +
        COALESCE(taxi_count,0) AS total_people
    FROM bike, subway, taxi;
    """

    params_specific = (date, hour, zone, date, hour, zone, date, hour, zone)
    specific_df = pd.read_sql(query_specific, conn, params=params_specific)
    specific_value = specific_df.iloc[0]["total_people"]

    # 2) Query for bucket averages
    query_buckets = """
    WITH citibike_sums AS (
        SELECT
            time_bucket(ct.hour) AS bucket,
            COUNT(*) AS citibike_count
        FROM public.citibike_trips ct
        JOIN public.citibike_zones cz
            ON ct.station_id = cz.station_id
        WHERE ct.date = %s
          AND cz.Zone = %s
        GROUP BY bucket
    ),
    mta_sums AS (
        SELECT
            time_bucket(mr.hour) AS bucket,
            SUM(mr.avg_ridership) AS mta_count
        FROM public.mta_ridership mr
        JOIN public.mta_zones mz
            ON mr.station_clean = mz.station_clean
        WHERE mr.date = %s
          AND mz.Zone = %s
        GROUP BY bucket
    ),
    taxi_sums AS (
        SELECT
            time_bucket(tr.hour) AS bucket,
            SUM(tr.ridership) AS taxi_count
        FROM public.taxi_ridership tr
        JOIN public.taxi_zones tz
            ON tr.LocationID = tz.LocationID
        WHERE tr.date = %s
          AND tz.Zone = %s
        GROUP BY bucket
    ),
    combined AS (
        SELECT
            COALESCE(c.bucket, m.bucket, t.bucket) AS bucket,
            COALESCE(c.citibike_count,0) +
            COALESCE(m.mta_count,0) +
            COALESCE(t.taxi_count,0) AS total_people
        FROM citibike_sums c
        FULL OUTER JOIN mta_sums m
          ON c.bucket = m.bucket
        FULL OUTER JOIN taxi_sums t
          ON COALESCE(c.bucket, m.bucket) = t.bucket
    )
    SELECT
        bucket,
        AVG(total_people)::numeric(12,2) AS avg_people
    FROM combined
    GROUP BY bucket
    ORDER BY
        CASE bucket
            WHEN 'Night' THEN 1
            WHEN 'Morning' THEN 2
            WHEN 'Afternoon' THEN 3
            WHEN 'Evening' THEN 4
        END;
    """

    params_buckets = (date, zone, date, zone, date, zone)
    buckets_df = pd.read_sql(query_buckets, conn, params=params_buckets)

    # Combine both results into a dictionary
    return {
        "specific_hour": specific_value,
        "bucket_averages": buckets_df
    }

In [67]:
result = get_ridership_info(conn, "06-01", 14, "Midtown South")
print("People at this exact hour:", result["specific_hour"])
print("\nBucketed averages:")
print(result["bucket_averages"])

C:\Users\maxim\AppData\Local\Temp\ipykernel_14828\881462952.py:47: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  specific_df = pd.read_sql(query_specific, conn, params=params_specific)
C:\Users\maxim\AppData\Local\Temp\ipykernel_14828\881462952.py:112: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  buckets_df = pd.read_sql(query_buckets, conn, params=params_buckets)


People at this exact hour: 2430.0

Bucketed averages:
      bucket  avg_people
0      Night      5660.0
1    Morning      8310.0
2  Afternoon     14534.0
3    Evening     21340.0
4    Unknown      1432.0


In [77]:
conn.rollback()
print("Transaction reset.")


Transaction reset.


In [78]:
def get_global_average(conn):
    """
    Computes the single global average across:
    - all zones
    - all dates
    - all time buckets
    combining Citibike + MTA + Taxi.
    Returns one scalar number.
    """
    
    query = """
    WITH citibike_sums AS (
        SELECT
            cz.Zone AS Zone,
            ct.date AS date,
            time_bucket(ct.hour) AS bucket,
            COUNT(*) AS citibike_count
        FROM public.citibike_trips ct
        JOIN public.citibike_zones cz
            ON ct.station_id = cz.station_id
        GROUP BY cz.Zone, ct.date, bucket
    ),

    mta_sums AS (
        SELECT
            mz.Zone AS Zone,
            mr.date AS date,
            time_bucket(mr.hour) AS bucket,
            SUM(mr.avg_ridership) AS mta_count
        FROM public.mta_ridership mr
        JOIN public.mta_zones mz
            ON mr.station_clean = mz.station_clean
        GROUP BY mz.Zone, mr.date, bucket
    ),

    taxi_sums AS (
        SELECT
            tz.Zone AS Zone,
            tr.date AS date,
            time_bucket(tr.hour) AS bucket,
            SUM(tr.ridership) AS taxi_count
        FROM public.taxi_ridership tr
        JOIN public.taxi_zones tz
            ON tr.LocationID = tz.LocationID
        GROUP BY tz.Zone, tr.date, bucket
    ),

    combined AS (
        SELECT
            COALESCE(c.Zone, m.Zone, t.Zone) AS Zone,
            COALESCE(c.date, m.date, t.date) AS date,
            COALESCE(c.bucket, m.bucket, t.bucket) AS bucket,
            COALESCE(c.citibike_count, 0)
                + COALESCE(m.mta_count, 0)
                + COALESCE(t.taxi_count, 0) AS total_people
        FROM citibike_sums c
        FULL OUTER JOIN mta_sums m
            ON c.Zone = m.Zone
           AND c.date = m.date
           AND c.bucket = m.bucket
        FULL OUTER JOIN taxi_sums t
            ON COALESCE(c.Zone, m.Zone) = t.Zone
           AND COALESCE(c.date, m.date) = t.date
           AND COALESCE(c.bucket, m.bucket) = t.bucket
    )

    SELECT AVG(total_people)::numeric(12,2) AS global_avg
    FROM combined;
    """

    with conn.cursor() as cur:
        cur.execute(query)
        result = cur.fetchone()[0]

    return float(result) if result is not None else None


# Example usage:
avg_value = get_global_average(conn)
print("Global average people across all zones, dates, and time buckets:", avg_value)


Global average people across all zones, dates, and time buckets: 10252.56


In [40]:
cur.close()
conn.close()